In [1]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [5]:
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip",'r') as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip",'r') as z:
    z.extractall(".")
    
files = os.listdir(""/kaggle/working/train/"")

In [6]:
files = os.listdir("/kaggle/working/train/")

In [ ]:
classes = ["cat", "dog"]

In [8]:
import shutil

os.makedirs("/kaggle/working/train/cat", exist_ok=True)
os.makedirs("/kaggle/working/train/dog", exist_ok=True)

for file in files:
    if file.startswith(classes[0]):
        shutil.move(f"/kaggle/working/train/{file}", 
                    f"/kaggle/working/train/cat/{file}")
    else:
        shutil.move(f"/kaggle/working/train/{file}", 
                    f"/kaggle/working/train/dog/{file}")

In [ ]:
classes = ["cat", "dog"]

In [ ]:
datagenAug = ImageDataGenerator(
    rotation_range=90,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

trainGen = datagenAug.flow_from_directory(
    "/kaggle/working/train/", classes=classes,
    target_size=(224, 224), class_mode="categorical",
    batch_size=32, subset="training")

testGen = datagenAug.flow_from_directory(
    "/kaggle/working/train/", classes=classes,
    target_size=(224, 224), class_mode="categorical",
    batch_size=32, subset="validation")

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False,  input_tensor=Input(shape=(224, 224, 3)))

for layer in vgg_model.layers[:-1]:
    layer.trainable = False
    
head = vgg_model.output
flatten = Flatten()(head)
fc = Dense(512, activation='relu')(flatten)
output = Dense(len(trainGen.class_indices), activation="softmax")(fc)

model = Model(inputs=vgg_model.input, outputs=output)